## About
This notebook walks through the analysis of the section `Taxonomic classification and comparative analysis of distinct microbial signatures`  


## Supplementary tables created
- Table S1: Number of samples sequenced per metro station 
- Table S2: Total reads per sample after removing human and mouse reads 
- Table S3: Total detected species across different species categories 
- Table S4: Number of samples used per Cities from MetaSUB 


In [1]:
import pandas as pd
import os
import copy
from tqdm import tqdm
import json

In [2]:
## Parameters
bounds = [0.97, 0.8] 
peri = 0.15
MIN_SAMPLES = 12

In [3]:
prevalence_list = [(0.1,"1"),(0.01,"01"),(0.001,"001"),(0.0001,"0001")]
#prevalence_list = [(0.001,"001"),]

In [4]:
DATASET = os.path.join(
    "..","data"
)
CHENNAI_DATASET = os.path.join(DATASET,"Chennai_data")
CORE_TAXA = os.path.join(
    DATASET,
    "MetaSUB_data",
)
RESULTS = os.path.join(
    "..","results"
)
SUPPLEMENTARY = os.path.join(RESULTS, "supplementary")
os.makedirs(SUPPLEMENTARY, exist_ok = True)


## Helper Function

In [5]:
def prevalence_extractor(df, bounds=[], col_not_to_include = []):
    taxa_df = copy.copy(df)
    taxa_list = [i for i in taxa_df.columns if i not in col_not_to_include]

    for i in taxa_list:
        taxa_df[i] = taxa_df[i] > 0
        taxa_df[i] = taxa_df[i] * 1

    v = taxa_df[taxa_list].sum() / taxa_df.shape[0]
    v = v.to_frame()
    v = v.reset_index()
    v.rename(columns={"index": "Species", 0: "prevalence"}, inplace=True)
    #v["Species"] = v["Species"].str.replace("prev_", "")
    v.sort_values('prevalence', ascending = False, inplace = True)
    core_df = v.query("prevalence > @bounds[0]")
    sub_core_df = v.query("prevalence > @bounds[1] and prevalence < @bounds[0]")
    peripheral_df = v.query("prevalence < @peri and prevalence > 0.00001")
    v = v.rename(columns={'prevalence':'prevalence'})
    return v, core_df, sub_core_df, peripheral_df

def merger_function(taxa_df, metadata_df):
    taxa_temp = copy.copy(taxa_df)
    taxa_meta = copy.copy(metadata_df)
    taxa_temp = taxa_temp.merge(
        taxa_meta, how="inner", left_on="Samples", right_on="uuid"
    )
    return taxa_temp

def filter_all_zero(df1, col_not_to_include=["Samples"]):
    df = copy.copy(df1)
    col_list = [i for i in df.columns if i not in col_not_to_include]
    temp = df[col_list].sum().to_frame()#.reset_index()
    temp.rename(columns={0: "non_zeros"}, inplace=True)
    #print("before ", df.shape)
    ## The name of the species which are not having any read mapped. These columns can be dropped
    df = df.drop(
        columns=list(temp.query("non_zeros<=0.000001").index.values)
    )  
    return df


## supplementary Table S1 : Number of samples sequenced per metro station 

In [6]:
chennai_metadata = pd.read_csv(os.path.join(CHENNAI_DATASET, 'metadata_processed.csv'))
chennai_metadata[["2021","2022"]] = pd.get_dummies(chennai_metadata['Year'])*1
chennai_metadata.head()

,Samples,DNA/RNA,sample type(N/P/E),location name,sublocation,select object/sampling place,select surface material,select ground level,Traffic at the sampling location,temperature,date and time,Sampling Date,Month,Year,is_kiosk,is_Rod,2021,2022
0,368257968,DNA,E,Washermenpet,Metro,Bannister,Metal,NaN,NaN,NaN,12:37,8/12/2021,Dec,2021,Other,Other,1,0
1,368273379,DNA,E,Washermenpet,Train,Rod,Metal,NaN,NaN,NaN,12:44,8/12/2021,Dec,2021,Other,Rod,1,0
2,368281532,DNA,E,Central,Metro,Ticket Counter,Ceramic,NaN,NaN,NaN,12:57,8/12/2021,Dec,2021,Other,Other,1,0
3,368281511,DNA,E,Central,Metro,Kiosk,Glass,NaN,NaN,NaN,13:00,8/12/2021,Dec,2021,Kiosk,Other,1,0
4,368281318,DNA,E,Central,Metro,Ticket Counter,Ceramic,NaN,NaN,NaN,13:17,8/12/2021,Dec,2021,Other,Other,1,0


In [7]:

supp_t_s1 = chennai_metadata[["location name","2021","2022"]].groupby(['location name']).sum()
total_row = supp_t_s1.sum().to_frame().T 
total_row['location name'] = 'Total'
supp_t_s1 = supp_t_s1.reset_index() 
supp_t_s1 = pd.concat([supp_t_s1.reset_index(drop=True), total_row], ignore_index=True)
supp_t_s1.to_csv(os.path.join(SUPPLEMENTARY, "supplementary_table_s1.csv"), index=False)

## supplementary Table S2 : Total reads per sample after removing human and mouse reads 
This is derived from the lof files of Kraken2 

In [8]:
supp_table_s2 = pd.read_csv(os.path.join(CHENNAI_DATASET, "Chennai_kraken2_log.csv"))
supp_table_s2.to_csv(os.path.join(SUPPLEMENTARY, "supplementary_table_s2.csv"), index=False)

# Analysis
## Chennai analysis

In [9]:
## Load the Chennai bracken results
chennai_species_df = pd.read_csv(os.path.join(CHENNAI_DATASET, 'bracken_species_non_human_processed.csv'))
chennai_species_df.set_index('Samples', inplace= True)

supplementary_s3 = {"Species Category":["Core","Sub-core","Peripheral", "Unique-core"],}

for threshold,folder in prevalence_list:
    print(f"Working with Threshold : {threshold}")
    ## Make the directories
    RESULT_threshold = os.path.join(RESULTS, 'microbial_signatures', folder)
    os.makedirs(RESULT_threshold, exist_ok = True)
    ## Chennai 
    s_list = chennai_species_df.columns
    chennai_species_df[s_list] = chennai_species_df[s_list].applymap(lambda x: 0 if x < threshold else x)
    chennai_species_df = filter_all_zero(chennai_species_df)
    full_prevalence_df, chennai_core_df, chennai_sub_core_df, chennai_peripheral_df = prevalence_extractor(chennai_species_df, bounds=bounds, col_not_to_include = ['Samples'])

    core_species_dict = {'Chennai': { 
                                    'core':list(chennai_core_df["Species"].values),
                                    'sub_core':list(chennai_sub_core_df["Species"].values),
                                    'peripheral':list(chennai_peripheral_df["Species"].values) 
                                  } 
                                  }
    full_prevalence_df.to_csv(os.path.join(RESULT_threshold,'species_prevalence.csv'),index=False)
    ## ROW
    METADATA = os.path.join(CORE_TAXA,"Filtered_complete_metadata.csv",)
    metadata_df = pd.read_csv(METADATA)
    metasub_species = pd.read_csv(os.path.join(CORE_TAXA, 'Filtered_bracken_species_non_human_processed.csv'))

    s_list = [i for i in metasub_species.columns if i != 'Samples']
    print('Core apply threshold')
    metasub_species[s_list] = metasub_species[s_list].applymap(lambda x: 0 if x < threshold else x)
    #metasub_species = metasub_species.apply(abd_threshold, axis=1)
    print('Core filter all zero')
    metasub_species = filter_all_zero(metasub_species)
    print('Core merge metadata')
    metasub_species = merger_function(metasub_species, metadata_df)
    
    ## Filter the cities based on number of samples
    vc = metasub_species['city'].value_counts()
    filtered_vc = vc[vc >= MIN_SAMPLES]
    
    print("Iterate through each cities")
    for city in tqdm(filtered_vc.index):
        city_species_df = metasub_species.query('city == @city')
        col_not_to_include=["Samples","uuid",
                            "core_project",
                            "project",
                            "city",
                            "surface_material",
                            "continent",
                            "surface_ontology_fine",
                            "control_type",]
        _, _core_df, _sub_core_df, _peripheral_df = prevalence_extractor(city_species_df, bounds=bounds, col_not_to_include = col_not_to_include)
        core_species_dict[city] = { 
                                        'core':list(_core_df["Species"].values),
                                        'sub_core':list(_sub_core_df["Species"].values),
                                        'peripheral':list(_peripheral_df["Species"].values) 
                                      } 
                                      
    
    with open(os.path.join(RESULT_threshold,'all_city_core.json'), 'w') as fp:
        json.dump(core_species_dict, fp)

    ## Get the unique core for Chennai
    other_city = [i for i in core_species_dict.keys() if i != 'Chennai']

    unique_taxas_dict = {}
    #for city_of_int in all_cities:
    city_of_int = 'Chennai'
    all_other_city_core_subcore_set = set()
    for city in other_city:
        #if city != city_of_int:
        all_other_city_core_subcore_set = all_other_city_core_subcore_set.union(
            core_species_dict[city]["core"]
        )
        all_other_city_core_subcore_set = all_other_city_core_subcore_set.union(
            core_species_dict[city]["sub_core"]
        )

    c_difference = list(
        set(core_species_dict[city_of_int]["core"]).difference(all_other_city_core_subcore_set)
    )
    unique_taxas_dict[city_of_int] = c_difference
    print(
        f" Total {city_of_int.capitalize()} {'core'}",
        len(core_species_dict[city_of_int]["core"]),
        "Where",
        c_difference.__len__(),
        "are unique to the city",
    )
    temp = pd.DataFrame.from_dict({"Chennai_spl": unique_taxas_dict["Chennai"]})
    temp.to_csv(os.path.join(RESULT_threshold, "chennai_spl_core_considered_core_subcore.csv"), index=False)

    supplementary_s3[f"Relative abundance > 0.{folder}"] = [len(core_species_dict["Chennai"]["core"]),
                                                            len(core_species_dict["Chennai"]["sub_core"]),
                                                            len(core_species_dict["Chennai"]["peripheral"]),
                                                            len(unique_taxas_dict["Chennai"]),]
pd.DataFrame.from_dict(supplementary_s3).to_csv(os.path.join(SUPPLEMENTARY, "supplementary_table_s3.csv"), index=False)
        

Working with Threshold : 0.1
Core apply threshold
Core filter all zero
Core merge metadata
No of MetaSUB sampled (2515, 237)
Iterate through each cities


100%|██████████| 31/31 [00:16<00:00,  1.93it/s]


 Total Chennai core 0 Where 0 are unique to the city
Working with Threshold : 0.01
Core apply threshold
Core filter all zero
Core merge metadata
No of MetaSUB sampled (2515, 1064)
Iterate through each cities


100%|██████████| 31/31 [00:57<00:00,  1.87s/it]


 Total Chennai core 0 Where 0 are unique to the city
Working with Threshold : 0.001
Core apply threshold
Core filter all zero
Core merge metadata
No of MetaSUB sampled (2515, 3085)
Iterate through each cities


100%|██████████| 31/31 [02:44<00:00,  5.31s/it]


 Total Chennai core 0 Where 0 are unique to the city
Working with Threshold : 0.0001
Core apply threshold
Core filter all zero
Core merge metadata
No of MetaSUB sampled (2515, 5824)
Iterate through each cities


100%|██████████| 31/31 [05:30<00:00, 10.65s/it]

 Total Chennai core 0 Where 0 are unique to the city


## supplementary s4 : Number of samples used per Cities from MetaSUB

In [15]:
## Actually 38, total sampled 2515
metasub_species['city'].value_counts().sum()


2515

In [20]:
filtered_vc.to_csv(os.path.join(SUPPLEMENTARY, "supplementary_table_s4.csv"))
## sum to get the total number 
print("No of MetaSUB sampled", filtered_vc.sum(), "cities", len(filtered_vc))

No of MetaSUB sampled 2461 cities 31
